In [ ]:
import pandas as pd
import re
import os

RESULTS_DIR = './../datasets_reduced/revision/results/'

RESULTS_CHAABEN = 'dataset_chaaben_baseline_with_completions.csv'
RESULTS_RAMC = 'results_with_comparable_info.csv' 
RESULTS_RANDOM_RAC = 'dataset_random_baseline_with_completions.csv'
OUTPUT_FILE = 'benchmark_results.csv' #'results_with_comparable_info.csv'

# cd with jupyter into the directory with the results
os.chdir(RESULTS_DIR)


DATASET_RESULTS_FILES = [RESULTS_RAMC, RESULTS_RANDOM_RAC]